In [3]:
from pdfminer.high_level import extract_text
import json
import os


In [4]:


def extract_text_from_pdf(pdf_path):
    try:
        text_content = {}
        text = extract_text(pdf_path)
        pages = text.split('\x0c')
        
        for page_num, page_text in enumerate(pages):
            text_content[str(page_num)] = page_text.strip()
            print(f"Extracted text from page {page_num}: {page_text[:100]}...")  # Print first 100 characters of the page text

        return text_content
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
        return None

if __name__ == "__main__":
    # Set the path to your PDF file directly
    pdf_path = r"C:\Users\Yadav Ji\Desktop\Data_Science.pdf"
    
    if not os.path.exists(pdf_path):
        print(f"File not found: {pdf_path}")
    else:
        text_content = extract_text_from_pdf(pdf_path)
        
        if text_content is not None:
            # Ensure the 'data' directory exists
            if not os.path.exists('data'):
                os.makedirs('data')
                print(f"Created directory 'data'")
            
            json_file_path = "data/text_content.json"
            with open(json_file_path, "w") as file:
                json.dump(text_content, file, indent=4)
                print(f"Text content extracted and saved to {json_file_path}")

            # Verify if the file was created and has content
            if os.path.exists(json_file_path):
                with open(json_file_path, "r") as file:
                    content = file.read()
                    if content.strip():
                        print(f"File {json_file_path} created successfully and is not empty")
                    else:
                        print(f"File {json_file_path} is empty")
        else:
            print("No text content extracted from the PDF")


Extracted text from page 0: ...
Extracted text from page 1: ...
Extracted text from page 2: Data	Science	from	Scratch

Joel	Grus

...
Extracted text from page 3: ...
Extracted text from page 4: Data	Science	from	Scratch

by	Joel	Grus

Copyright	©	2015	O’Reilly	Media.	All	rights	reserved.

Prin...
Extracted text from page 5: Revision	History	for	the	First	Edition

2015-04-10:	First	Release

See	http://oreilly.com/catalog/er...
Extracted text from page 6: ...
Extracted text from page 7: Preface

...
Extracted text from page 8: Data	Science

Data	scientist	has	been	called	“the	sexiest	job	of	the	21st	century,”	presumably	by
so...
Extracted text from page 9: From	Scratch

There	are	lots	and	lots	of	data	science	libraries,	frameworks,	modules,	and	toolkits	t...
Extracted text from page 10: Over	the	years,	I’ve	trained	a	number	of	data	scientists.	While	not	all	of	them	have	gone
on	to	beco...
Extracted text from page 11: Conventions	Used	in	This	Book

The	following	typographical	conventions	

In [5]:
import json
import os



In [6]:


class Node:
    def __init__(self, identifier, content):
        self.id = identifier
        self.content = content
        self.children = []

def create_hierarchical_tree(text_content):
    root = Node("root", "Entire Textbook")
    
    # This is a simplified example. You would parse the text_content to identify chapters and sections.
    # Let's assume each page is a chapter for this example
    for page_num, page_text in text_content.items():
        chapter = Node(f"chapter_{page_num}", f"Chapter {int(page_num) + 1}")
        section = Node(f"section_{page_num}_1", f"Section {int(page_num) + 1}.1")
        section.children.append(Node(f"paragraph_{page_num}_1_1", page_text))
        
        chapter.children.append(section)
        root.children.append(chapter)
    
    return root

if __name__ == "__main__":
    json_file_path = "data/text_content.json"
    
    if not os.path.exists(json_file_path):
        print(f"File not found: {json_file_path}")
    else:
        with open(json_file_path, "r") as file:
            text_content = json.load(file)
        
        tree = create_hierarchical_tree(text_content)
        
        # Save the tree to a file
        tree_file_path = "data/hierarchical_tree.json"
        with open(tree_file_path, "w") as file:
            json.dump(tree.__dict__, file, default=lambda o: o.__dict__, indent=4)
        print(f"Hierarchical tree created and saved to {tree_file_path}")


Hierarchical tree created and saved to data/hierarchical_tree.json


In [7]:
from rank_bm25 import BM25Okapi
from transformers import DPRQuestionEncoderTokenizer, DPRQuestionEncoder, DPRContextEncoder, DPRContextEncoderTokenizer
import json
import numpy as np



ModuleNotFoundError: No module named 'rank_bm25'

In [17]:


def load_tree(tree_file_path):
    with open(tree_file_path, "r") as file:
        tree = json.load(file)
    return tree

def traverse_tree(node, query, depth=0):
    if not node["children"]:
        return [(node["content"], depth)]
    results = []
    for child in node["children"]:
        results.extend(traverse_tree(child, query, depth + 1))
    return results

def retrieve_relevant_sections(tree, query):
    root = tree
    all_sections = traverse_tree(root, query)
    all_sections.sort(key=lambda x: x[1])  # Sort by depth
    return [section[0] for section in all_sections]

def dpr_retrieval(corpus, query):
    question_encoder = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
    question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')

    context_encoder = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
    context_tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')

    question_embedding = question_encoder(**question_tokenizer(query, return_tensors='pt', truncation=True, padding=True, max_length=512))[0].detach().numpy()
    context_embeddings = np.array([
        context_encoder(**context_tokenizer(doc, return_tensors='pt', truncation=True, padding=True, max_length=512))[0].detach().numpy()
        for doc in corpus
    ])

    similarities = np.dot(context_embeddings, question_embedding.T).squeeze()
    return np.argsort(similarities)[::-1], np.sort(similarities)[::-1]

if __name__ == "__main__":
    tree_file_path = "data/hierarchical_tree.json"
    tree = load_tree(tree_file_path)
    
    query = "Your query here"
    relevant_sections = retrieve_relevant_sections(tree, query)
    
    # BM25 Retrieval (assuming you have implemented it elsewhere)
    bm25_indices, bm25_scores = bm25_retrieval(corpus, query)
    print("BM25 Results:", bm25_indices, bm25_scores)

    # DPR Retrieval
    dpr_indices, dpr_scores = dpr_retrieval(relevant_sections, query)
    print("DPR Results:", dpr_indices, dpr_scores)



BM25 Results: [464 158 146 147 148 149 150 151 152 153 154 155 156 157 159 231 160 161
 162 163 164 165 166 167 168 169 170 171 145 144 143 142 117 118 119 120
 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138
 139 140 141 172 173 174 203 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 204 202 175 201
 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193
 194 195 196 197 198 199 200 116 115 114  28  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  29  27  56  26   1   2   3   4   5   6   7   8   9  10  11  12  13  14
  15  16  17  18  19  20  21  22  23  24  25  55  57 113  86  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112  87  85  58  84  59  60  61  62  63  64  65  66  67  68
  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83 230 232 463
 391 379 380 381 382 383 384 385 386 

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the

KeyboardInterrupt: 

In [8]:
from transformers import pipeline
import json
import os

def load_tree(tree_file_path):
    with open(tree_file_path, "r") as file:
        tree = json.load(file)
    return tree

def traverse_tree(node, query, depth=0):
    if not node["children"]:
        return [(node["content"], depth)]
    results = []
    for child in node["children"]:
        results.extend(traverse_tree(child, query, depth + 1))
    return results

def retrieve_relevant_sections(tree, query):
    root = tree
    all_sections = traverse_tree(root, query)
    all_sections.sort(key=lambda x: x[1])  # Sort by depth
    return [section[0] for section in all_sections]

if __name__ == "__main__":
    tree_file_path = "data/hierarchical_tree.json"
    tree = load_tree(tree_file_path)
    
    query = "Your query here"
    relevant_sections = retrieve_relevant_sections(tree, query)
    
    # Use a language model to generate answers
    qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")
    answers = [qa_pipeline(question=query, context=section) for section in relevant_sections]
    
    for answer in answers:
        print(f"Answer: {answer['answer']}, Score: {answer['score']}")


ModuleNotFoundError: No module named 'transformers'

In [15]:
import streamlit as st
from transformers import pipeline
import json

def load_tree(tree_file_path):
    with open(tree_file_path, "r") as file:
        tree = json.load(file)
    return tree

def traverse_tree(node, query, depth=0):
    if not node["children"]:
        return [(node["content"], depth)]
    results = []
    for child in node["children"]:
        results.extend(traverse_tree(child, query, depth + 1))
    return results

def retrieve_relevant_sections(tree, query):
    root = tree
    all_sections = traverse_tree(root, query)
    all_sections.sort(key=lambda x: x[1])  # Sort by depth
    return [section[0] for section in all_sections]

st.title("Textbook QA System")

query = st.text_input("Enter your query:")
if query:
    tree_file_path = "data/hierarchical_tree.json"
    tree = load_tree(tree_file_path)
    
    relevant_sections = retrieve_relevant_sections(tree, query)
    
    # Use a language model to generate answers
    qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")
    answers = [qa_pipeline(question=query, context=section) for section in relevant_sections]
    
    for answer in answers:
        st.write(f"Answer: {answer['answer']}, Score: {answer['score']}")


2024-07-16 10:13:27.191 
  command:

    streamlit run C:\anaconda\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
